## MDS201803<BR>MDS201811

## Importing and Preprocessing the data

In [1]:
import json    
raw_data = []
with open(r"C:\Users\LENOVO\Downloads\SEM2\DMML\Assignments\News-Classification-DataSet\News_Classification_DataSet.json") as f:
    for line in f:
        raw_data.append(json.loads(line))

Parsing the data to extract the news contents & labels

In [2]:
contents=[]
for i in range(0,7600):
    contents.append(raw_data[i]["content"])
label=[]
for i in range(0,7600):
    label.append(raw_data[i]["annotation"]["label"])    

In [3]:
import numpy as np
label = np.array(list(label))

In [4]:
uniq_label, converted_array_label = np.unique(label, return_inverse=True) #Indexing the labels

In [5]:
contents = np.array(list(contents)) # Creating a numpy array of the contents

We vectorize the news contents and create a sparse matrix 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = contents
vectorizer = CountVectorizer()
new_content=vectorizer.fit_transform(corpus).todense().tolist()
new_content_array=np.array(list(new_content))

### Train-Test Split of the data in 90:10 ratio

In [7]:
from sklearn.model_selection import train_test_split
label_train,label_test,content_train,content_test=train_test_split(converted_array_label,new_content_array,test_size=0.1,random_state=60)

We vectorize the news labels with "one-hot" encoding

In [8]:
def to_one_hot(labels, dimension=4):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Our vectorized training labels
one_hot_train_labels = to_one_hot(label_train)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(label_test)

In [9]:
from keras.utils.np_utils import to_categorical
one_hot_train_labels = to_categorical(label_train)
one_hot_test_labels = to_categorical(label_test)

Using TensorFlow backend.


After parsing and Train-test split we have the data as follows:(With 20031 unique words in the contents and 4 unique labels)<br>
So, our input is a (20031 x 1) boolean vector which correspond to a perticular news content.<br>
And, our output is a (4 x 1) boolean vector which correspond to a perticular news lebel.

In [10]:
print(one_hot_train_labels.shape) #Train labels
print(one_hot_test_labels.shape) # Test labels
print(content_train.shape) #Train contents
print(content_test.shape) #Test contents

(6840, 4)
(760, 4)
(6840, 20031)
(760, 20031)


# Calculating CV score (10-fold)

In [11]:
from keras import models
from keras import layers
from sklearn.model_selection import cross_val_score
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

## Model 1: Using Sigmoid as activation function at all layers

In [12]:
def create_network_1():
    model = models.Sequential()
    model.add(layers.Dense(40, activation='sigmoid', input_shape=(20031,)))
    model.add(layers.Dense(40, activation='sigmoid'))
    model.add(layers.Dense(4, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

neural_network = KerasClassifier(build_fn=create_network_1, 
                                 epochs=10, 
                                 batch_size=512, 
                                 verbose=0)

In [13]:
cross_val_score(neural_network,content_train,one_hot_train_labels, cv=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


array([0.84064327, 0.86695907, 0.85672514, 0.83333333, 0.82309942,
       0.82602339, 0.85818713, 0.81432748, 0.86403509, 0.83040936])

In [33]:
model = models.Sequential()
model.add(layers.Dense(40, activation='sigmoid', input_shape=(20031,)))
model.add(layers.Dense(40, activation='sigmoid'))
model.add(layers.Dense(4, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(content_train,one_hot_train_labels,epochs = 10,batch_size = 512)
results = model.evaluate(content_test,one_hot_test_labels)
print(results)

Epoch 1/10
6840/6840 [==============================] - 3s 474us/step - loss: 1.3997 - acc: 0.2519
Epoch 2/10
6840/6840 [==============================] - 2s 333us/step - loss: 1.3456 - acc: 0.3876
Epoch 3/10
6840/6840 [==============================] - 2s 300us/step - loss: 1.3018 - acc: 0.8069
Epoch 4/10
6840/6840 [==============================] - 2s 290us/step - loss: 1.2394 - acc: 0.8637
Epoch 5/10
6840/6840 [==============================] - 2s 308us/step - loss: 1.1604 - acc: 0.8762
Epoch 6/10
6840/6840 [==============================] - 2s 299us/step - loss: 1.0727 - acc: 0.8772
Epoch 7/10
6840/6840 [==============================] - 2s 304us/step - loss: 0.9813 - acc: 0.8902
Epoch 8/10
6840/6840 [==============================] - 2s 308us/step - loss: 0.8902 - acc: 0.8944
Epoch 9/10
6840/6840 [==============================] - 2s 277us/step - loss: 0.8015 - acc: 0.9028
Epoch 10/10
760/760 [==============================] - 1s 1ms/step
[0.7746408493895279, 0.8236842098988985]


In [32]:
label_pred_1 = model.predict(content_test)
label_pred_1 = np.argmax(label_pred_1, axis=1)
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(label_test, label_pred_1, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Precision:  0.8400154147966452
Recall:  0.8408875957348906
F1 score:  0.838879032179674


## Model 2: Using SeLU as activation function and softmax at output layer

In [17]:
def create_network_2():
    model = models.Sequential()
    model.add(layers.Dense(40, activation='selu', input_shape=(20031,)))
    model.add(layers.Dense(40, activation='selu'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

neural_network = KerasClassifier(build_fn=create_network_2, 
                                 epochs=10, 
                                 batch_size=512, 
                                 verbose=0)

In [18]:
cross_val_score(neural_network,content_train,one_hot_train_labels, cv=10)

array([0.875731  , 0.86111111, 0.85526316, 0.86695906, 0.86549707,
       0.84064328, 0.86695906, 0.82748538, 0.86842105, 0.8494152 ])

In [34]:
model = models.Sequential()
model.add(layers.Dense(40, activation='selu', input_shape=(20031,)))
model.add(layers.Dense(40, activation='selu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(content_train,one_hot_train_labels,epochs = 10,batch_size = 512)
results = model.evaluate(content_test,one_hot_test_labels)
print(results)

Epoch 1/10
6840/6840 [==============================] - 4s 587us/step - loss: 0.8606 - acc: 0.7586
Epoch 2/10
6840/6840 [==============================] - 2s 279us/step - loss: 0.3461 - acc: 0.9231
Epoch 3/10
6840/6840 [==============================] - 2s 277us/step - loss: 0.1939 - acc: 0.9585
Epoch 4/10
6840/6840 [==============================] - 2s 293us/step - loss: 0.1113 - acc: 0.9784
Epoch 5/10
6840/6840 [==============================] - 2s 293us/step - loss: 0.0611 - acc: 0.9905
Epoch 6/10
6840/6840 [==============================] - 2s 278us/step - loss: 0.0324 - acc: 0.9961
Epoch 7/10
6840/6840 [==============================] - 2s 275us/step - loss: 0.0173 - acc: 0.9978
Epoch 8/10
6840/6840 [==============================] - 2s 296us/step - loss: 0.0093 - acc: 0.9993
Epoch 9/10
6840/6840 [==============================] - 2s 292us/step - loss: 0.0049 - acc: 0.9996
Epoch 10/10
760/760 [==============================] - 1s 1ms/step
[0.6266005139601858, 0.8539473684210527]


In [35]:
label_pred_2 = model.predict(content_test)
label_pred_2 = np.argmax(label_pred_2, axis=1)
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(label_test, label_pred_2, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Precision:  0.8545691266786422
Recall:  0.8558938919379728
F1 score:  0.8549979901282609


## Model 3: Using Sigmoid as activation function and softmax in output layer

In [25]:
def create_network_3():
    model = models.Sequential()
    model.add(layers.Dense(40, activation='sigmoid', input_shape=(20031,)))
    model.add(layers.Dense(40, activation='sigmoid'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

neural_network = KerasClassifier(build_fn=create_network_3, 
                                 epochs=10, 
                                 batch_size=512, 
                                 verbose=0)

In [26]:
cross_val_score(neural_network,content_train,one_hot_train_labels, cv=10)

array([0.81871345, 0.86988304, 0.83625731, 0.85672514, 0.84210526,
       0.82309941, 0.86695906, 0.83040935, 0.85818713, 0.83040935])

In [37]:
model = models.Sequential()
model.add(layers.Dense(40, activation='sigmoid', input_shape=(20031,)))
model.add(layers.Dense(40, activation='sigmoid'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(content_train,one_hot_train_labels,epochs = 10,batch_size = 512)
results = model.evaluate(content_test,one_hot_test_labels)
print(results)

Epoch 1/10
6840/6840 [==============================] - 4s 593us/step - loss: 1.3652 - acc: 0.4183
Epoch 2/10
6840/6840 [==============================] - 2s 295us/step - loss: 1.3024 - acc: 0.7392
Epoch 3/10
6840/6840 [==============================] - 2s 280us/step - loss: 1.2287 - acc: 0.8212
Epoch 4/10
6840/6840 [==============================] - 2s 280us/step - loss: 1.1380 - acc: 0.8635
Epoch 5/10
6840/6840 [==============================] - 2s 289us/step - loss: 1.0361 - acc: 0.8844
Epoch 6/10
6840/6840 [==============================] - 2s 300us/step - loss: 0.9269 - acc: 0.8905
Epoch 7/10
6840/6840 [==============================] - 2s 290us/step - loss: 0.8173 - acc: 0.9001
Epoch 8/10
6840/6840 [==============================] - 2s 298us/step - loss: 0.7140 - acc: 0.9042
Epoch 9/10
6840/6840 [==============================] - 2s 286us/step - loss: 0.6212 - acc: 0.9120
Epoch 10/10
760/760 [==============================] - 1s 1ms/step
[0.6285303285247401, 0.8565789473684211]


In [38]:
label_pred_3 = model.predict(content_test)
label_pred_3 = np.argmax(label_pred_3, axis=1)
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(label_test, label_pred_2, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Precision:  0.8545691266786422
Recall:  0.8558938919379728
F1 score:  0.8549979901282609


## Model 4: Using ReLU as activation function and softmax in output layer

In [40]:
def create_network_4():
    model = models.Sequential()
    model.add(layers.Dense(40, activation='relu', input_shape=(20031,)))
    model.add(layers.Dense(40, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

neural_network = KerasClassifier(build_fn=create_network_4, 
                                 epochs=10, 
                                 batch_size=512, 
                                 verbose=0)

In [41]:
cross_val_score(neural_network,content_train,one_hot_train_labels, cv=10)

array([0.86842105, 0.875731  , 0.86842105, 0.87573099, 0.87573099,
       0.84941521, 0.86988304, 0.84356725, 0.86111111, 0.85380117])

In [42]:
model = models.Sequential()
model.add(layers.Dense(40, activation='relu', input_shape=(20031,)))
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(content_train,one_hot_train_labels,epochs = 10,batch_size = 512)
results = model.evaluate(content_test,one_hot_test_labels)
print(results)

Epoch 1/10
6840/6840 [==============================] - 5s 753us/step - loss: 1.1488 - acc: 0.7105
Epoch 2/10
6840/6840 [==============================] - 2s 297us/step - loss: 0.6323 - acc: 0.9061
Epoch 3/10
6840/6840 [==============================] - 2s 295us/step - loss: 0.3779 - acc: 0.9336
Epoch 4/10
6840/6840 [==============================] - 2s 278us/step - loss: 0.2450 - acc: 0.9538
Epoch 5/10
6840/6840 [==============================] - 2s 300us/step - loss: 0.1634 - acc: 0.9686
Epoch 6/10
6840/6840 [==============================] - 2s 326us/step - loss: 0.1069 - acc: 0.9817
Epoch 7/10
6840/6840 [==============================] - 2s 284us/step - loss: 0.0693 - acc: 0.9885
Epoch 8/10
6840/6840 [==============================] - 2s 314us/step - loss: 0.0431 - acc: 0.9943
Epoch 9/10
6840/6840 [==============================] - 2s 295us/step - loss: 0.0262 - acc: 0.9965
Epoch 10/10
760/760 [==============================] - 1s 2ms/step
[0.5215951213711186, 0.8552631578947368]


In [43]:
label_pred_4 = model.predict(content_test)
label_pred_4 = np.argmax(label_pred_4, axis=1)
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(label_test, label_pred_2, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Precision:  0.8545691266786422
Recall:  0.8558938919379728
F1 score:  0.8549979901282609


## Model 5: Using tanH as activation function and softmax in output layer

In [44]:
def create_network_5():
    model = models.Sequential()
    model.add(layers.Dense(40, activation='tanh', input_shape=(20031,)))
    model.add(layers.Dense(40, activation='tanh'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

neural_network = KerasClassifier(build_fn=create_network_5, 
                                 epochs=10, 
                                 batch_size=512, 
                                 verbose=0)

In [45]:
cross_val_score(neural_network,content_train,one_hot_train_labels, cv=10)

array([0.87134503, 0.86695907, 0.87280702, 0.87134503, 0.88304093,
       0.84502924, 0.86988304, 0.83333333, 0.86842105, 0.84502923])

In [46]:
model = models.Sequential()
model.add(layers.Dense(40, activation='tanh', input_shape=(20031,)))
model.add(layers.Dense(40, activation='tanh'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(content_train,one_hot_train_labels,epochs = 10,batch_size = 512)
results = model.evaluate(content_test,one_hot_test_labels)
print(results)

Epoch 1/10
6840/6840 [==============================] - 5s 722us/step - loss: 0.9577 - acc: 0.7579
Epoch 2/10
6840/6840 [==============================] - 2s 292us/step - loss: 0.4687 - acc: 0.9054
Epoch 3/10
6840/6840 [==============================] - 2s 278us/step - loss: 0.2917 - acc: 0.9382
Epoch 4/10
6840/6840 [==============================] - 2s 290us/step - loss: 0.1908 - acc: 0.9592
Epoch 5/10
6840/6840 [==============================] - 2s 283us/step - loss: 0.1242 - acc: 0.9747
Epoch 6/10
6840/6840 [==============================] - 2s 282us/step - loss: 0.0790 - acc: 0.9861
Epoch 7/10
6840/6840 [==============================] - 2s 299us/step - loss: 0.0490 - acc: 0.9923
Epoch 8/10
6840/6840 [==============================] - 2s 299us/step - loss: 0.0291 - acc: 0.9969
Epoch 9/10
6840/6840 [==============================] - 2s 276us/step - loss: 0.0175 - acc: 0.9982
Epoch 10/10
760/760 [==============================] - 1s 2ms/step
[0.5286685074630536, 0.8578947368421053]


In [47]:
label_pred_5 = model.predict(content_test)
label_pred_5 = np.argmax(label_pred_5, axis=1)
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(label_test, label_pred_2, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Precision:  0.8545691266786422
Recall:  0.8558938919379728
F1 score:  0.8549979901282609
